In [1]:
import os
import sys
sys.path.append(os.path.abspath(".."))
import pickle
import numpy as np
import pandas as pd
import copy
from tqdm.autonotebook import tqdm
import bayesianssa

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:8: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  


### Preprocess

In [2]:
parser = bayesianssa.ReactionFormulaParser(
    f'../data/SSDesignSuppTable.csv',
    ignored_reactions=['BIO'],
    ignored_metabolites=['Export'])
parser.parse()
nu = parser.nu
nu = bayesianssa.remove_non_flow_metabolites(nu)

H[c], NH3[c] do not flow.


### Experimental settings

In [3]:
n_iter = 10000
a = 3
b = 1

### Initial calculation

In [4]:
indices = np.argwhere(nu.T.to_numpy() < 0)
np.random.seed(0)
init_bssa = bayesianssa.BayesianSSA(nu, indices, a, b,
                                    n_iter=n_iter, verbose=False)
init_bssa.run()

  0%|          | 0/10000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [5]:
with open(f'../results/ssa_results.pkl', 'wb') as f:
    pickle.dump(init_bssa.Q_consensus, f)

In [6]:
with open(f'../results/initial_confs.pkl', 'wb') as f:
    pickle.dump(init_bssa.confidence, f)

### Synthetic data generation

In [7]:
Ns = 30

In [8]:
def generate_data(Ns, perturbed_reactions, is_pq_reversed,
                  p=0.1, q=0.1, random_seed=0):
    np.random.seed(random_seed)
    Nr = len(perturbed_reactions)
    if is_pq_reversed == None:
        is_pq_reversed = [False] * Nr
    rhos = []
    for i in range(Nr):
        if is_pq_reversed[i]:
            rho = np.random.beta(q, p)
        else:
            rho = np.random.beta(p, q)
        rhos.append(rho)
    generated_data = []
    for rho, perturbed_reaction in zip(rhos, perturbed_reactions):
        for _ in range(Ns):
            x = np.random.choice([1, 0], p=[rho, 1 - rho])
            generated_data.append(['SUCCt', perturbed_reaction, x])
    generated_data = pd.DataFrame(generated_data).sample(frac=1).to_numpy()
    return perturbed_reactions, rhos, generated_data

In [9]:
perturbed_reactions = ['GND', 'PTS', 'PPC']
is_pq_reversed=[True, True, False]

In [10]:
perturbed_reactions, rhos, generated_data = generate_data(
    Ns, perturbed_reactions, is_pq_reversed,
    p=0.3, q=0.1, random_seed=0)

In [11]:
rhos

[0.0075202575054505744, 0.04572269439620904, 0.8188253470361212]

In [12]:
generated_data

array([['SUCCt', 'PTS', 0],
       ['SUCCt', 'PPC', 1],
       ['SUCCt', 'PTS', 0],
       ['SUCCt', 'PPC', 1],
       ['SUCCt', 'GND', 0],
       ['SUCCt', 'PPC', 1],
       ['SUCCt', 'PTS', 0],
       ['SUCCt', 'PPC', 1],
       ['SUCCt', 'PTS', 0],
       ['SUCCt', 'GND', 0],
       ['SUCCt', 'PPC', 1],
       ['SUCCt', 'GND', 0],
       ['SUCCt', 'PPC', 0],
       ['SUCCt', 'PPC', 1],
       ['SUCCt', 'GND', 0],
       ['SUCCt', 'PTS', 0],
       ['SUCCt', 'PTS', 0],
       ['SUCCt', 'GND', 0],
       ['SUCCt', 'PTS', 0],
       ['SUCCt', 'GND', 0],
       ['SUCCt', 'PPC', 1],
       ['SUCCt', 'GND', 0],
       ['SUCCt', 'PPC', 1],
       ['SUCCt', 'GND', 0],
       ['SUCCt', 'GND', 0],
       ['SUCCt', 'GND', 0],
       ['SUCCt', 'PPC', 1],
       ['SUCCt', 'PTS', 0],
       ['SUCCt', 'GND', 0],
       ['SUCCt', 'GND', 0],
       ['SUCCt', 'PTS', 0],
       ['SUCCt', 'PTS', 0],
       ['SUCCt', 'PTS', 0],
       ['SUCCt', 'PPC', 1],
       ['SUCCt', 'PTS', 0],
       ['SUCCt', 'PP

In [13]:
pd.DataFrame(generated_data).to_csv('../data/synthetic_perturbation_data.csv')

### Synthetic data experiments

In [14]:
def evaluate_pred_prob(mth, calc_and_update_func, params, p_data):
    pred_probs = []
    for i, p_record in enumerate(tqdm(p_data)):
        this_pred_prob = calc_and_update_func(mth, params, p_record)
        pred_probs.append(this_pred_prob)
    return pred_probs

In [15]:
def calc_and_update_BSSA(mth, params, p_record):
    mth.a = params[0]
    mth.b = params[1]
    pred_prob = mth.calculate_predictive_prob(*p_record)
    ex_results = {}
    ex_results['target'] = [[p_record[0], p_record[1]]]
    ex_results['up/down'] = [p_record[2]]
    mth.update_distributions(ex_results)
    return pred_prob

In [16]:
def not_update_base_method(mth, params, p_record):
    pred_prob = mth.calculate_predictive_prob(*p_record)
    return pred_prob

In [17]:
gen_pred_probs = {}

In [18]:
params_list = [(9, 1), (6, 2), (3, 1), (2, 1)]

In [19]:
for params in params_list:
    bssa = copy.deepcopy(init_bssa)
    gen_pred_probs[f'BayesianSSA {params}'] = evaluate_pred_prob(
        bssa, calc_and_update_BSSA, params, generated_data)

  0%|          | 0/90 [00:00<?, ?it/s]

  0%|          | 0/90 [00:00<?, ?it/s]

  0%|          | 0/90 [00:00<?, ?it/s]

  0%|          | 0/90 [00:00<?, ?it/s]

In [20]:
base_method = copy.deepcopy(init_bssa)
gen_pred_probs['Base method'] = evaluate_pred_prob(
    base_method, not_update_base_method, None, generated_data)

  0%|          | 0/90 [00:00<?, ?it/s]

In [21]:
def return_random(data):
    return [0.5 for _ in range(len(data))]

In [22]:
gen_pred_probs['Random'] = return_random(generated_data)

In [23]:
def pred_probs_to_losses(pred_probs):
    loss = 0
    losses = []
    losses.append(loss)
    for n, pred_prob in enumerate(pred_probs):
        loss -= np.log(pred_prob)
        losses.append(loss)
    return losses

In [24]:
gen_losses = {}
method_names = [
                'BayesianSSA (9, 1)', 
                'BayesianSSA (6, 2)', 
                'BayesianSSA (3, 1)', 
                'BayesianSSA (2, 1)', 
                'Base method', 
                'Random'
]
for method_name in method_names:
    gen_losses[method_name] = pred_probs_to_losses(gen_pred_probs[method_name])

pd.DataFrame(gen_losses).to_csv('../results/synth_cross_entropy.csv')

### Pseudo data experiments

In [25]:
p_mth = copy.deepcopy(init_bssa)

In [26]:
ex_results = {}
ex_results['target'] = [['SUCCt', 'GND']] * 10
ex_results['up/down'] = [0] * 10
p_mth.update_distributions(ex_results)
p_mth.run()

In [27]:
with open(f'../results/gnd_updated_confs.pkl', 'wb') as f:
    pickle.dump(p_mth.confidence, f)

In [28]:
ex_results = {}
ex_results['target'] = [['SUCCt', 'PTS']] * 10
ex_results['up/down'] = [0] * 10
p_mth.update_distributions(ex_results)
p_mth.run()

In [29]:
with open(f'../results/gnd_pts_updated_confs.pkl', 'wb') as f:
    pickle.dump(p_mth.confidence, f)

##  Real data experiments

###  Predictive performance

#### Trajectory

In [30]:
class naive_bayes():
    def __init__(self, a, b):
        self.n = {}
        self.a = a
        self.b = b

    def update_distributions(self, ex_results):
        for up_down, (row_name, col_name) in zip(ex_results['up/down'], 
                                                 ex_results['target']):
            # first observation of (m, j) results
            if not (row_name, col_name) in self.n.keys():
                self.n[(row_name, col_name)] = {'positive': 0, 'negative': 0}
            if up_down == 1:
                self.n[(row_name, col_name)]['positive'] += 1
            elif up_down == 0:
                self.n[(row_name, col_name)]['negative'] += 1

    def calculate_predictive_prob(self, row_name, col_name, pos_or_neg):
        if (row_name, col_name) in self.n.keys():
            a_hat = self.a + self.n[(row_name, col_name)]['positive']
            b_hat = self.b + self.n[(row_name, col_name)]['negative']
        else:
            a_hat = self.a
            b_hat = self.b
        if pos_or_neg == 1:
            return a_hat / (a_hat + b_hat)
        if pos_or_neg == 0:
            return b_hat / (a_hat + b_hat)

In [31]:
real_data = pd.read_csv('../data/real_perturbation_data.csv').sample(frac=1).to_numpy()

In [32]:
real_data

array([['SUCCt', 'ME2', 0],
       ['SUCCt', 'FBP', 0],
       ['SUCCt', 'PTA', 0],
       ['SUCCt', 'PPC', 0],
       ['SUCCt', 'PTA', 0],
       ['SUCCt', 'ME1', 0],
       ['SUCCt', 'ME2', 0],
       ['SUCCt', 'LDH', 0],
       ['SUCCt', 'CS', 1],
       ['SUCCt', 'PPS', 0],
       ['SUCCt', 'ICL', 1],
       ['SUCCt', 'PPS', 0],
       ['SUCCt', 'FBP', 0],
       ['SUCCt', 'PPC', 0],
       ['SUCCt', 'FBP', 0],
       ['SUCCt', 'CS', 1],
       ['SUCCt', 'ME2', 0],
       ['SUCCt', 'CS', 0],
       ['SUCCt', 'ICL', 1],
       ['SUCCt', 'PPC', 0],
       ['SUCCt', 'PCK', 1],
       ['SUCCt', 'ME1', 0],
       ['SUCCt', 'LDH', 0],
       ['SUCCt', 'LDH', 0],
       ['SUCCt', 'PCK', 1],
       ['SUCCt', 'ICL', 1],
       ['SUCCt', 'PCK', 1],
       ['SUCCt', 'PPS', 1],
       ['SUCCt', 'PTA', 1],
       ['SUCCt', 'ME1', 0]], dtype=object)

In [33]:
r_pred_probs = {}

In [34]:
params_list = [(9, 1), (6, 2), (3, 1), (2, 1)]

In [35]:
for params in params_list:
    bssa = copy.deepcopy(init_bssa)
    r_pred_probs[f'BayesianSSA {params}'] = evaluate_pred_prob(
        bssa, calc_and_update_BSSA, params, real_data)

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

In [36]:
base_method = copy.deepcopy(init_bssa)
r_pred_probs['Base method'] = evaluate_pred_prob(
    base_method, not_update_base_method, None, real_data)

  0%|          | 0/30 [00:00<?, ?it/s]

In [37]:
nb_model = naive_bayes(1, 1)
params = (1, 1)
r_pred_probs['Naive Bayes'] = evaluate_pred_prob(
    nb_model, calc_and_update_BSSA, params, real_data)

  0%|          | 0/30 [00:00<?, ?it/s]

In [38]:
r_pred_probs['Random'] = return_random(real_data)

In [39]:
r_losses = {}
method_names = [
                'BayesianSSA (9, 1)', 
                'BayesianSSA (6, 2)', 
                'BayesianSSA (3, 1)', 
                'BayesianSSA (2, 1)', 
                'Base method', 
                'Naive Bayes', 
                'Random'
]
for method_name in method_names:
    r_losses[method_name] = pred_probs_to_losses(r_pred_probs[method_name])

In [40]:
pd.DataFrame(r_losses).to_csv('../results/real_cross_entropy.csv')

#### Out of sample

In [41]:
real_data_df = pd.DataFrame(real_data)
real_data_df.columns = ['Observation', 'Perturbation', 'Result']
perturbed_reactions = real_data_df['Perturbation'].unique().tolist()

In [42]:
def fit_bayesianssa(model, data):
    ex_results = {}
    ex_results['target'] = data[['Observation', 'Perturbation']].to_numpy()
    ex_results['up/down'] = data['Result'].to_numpy()
    model.update_distributions(ex_results)

In [43]:
pred_probs = []
for perturbed_reaction in perturbed_reactions:
    train_df = real_data_df[real_data_df['Perturbation'] != perturbed_reaction].copy()
    test_df = real_data_df[real_data_df['Perturbation'] == perturbed_reaction].copy()
    o_mth = copy.deepcopy(init_bssa)
    fit_bayesianssa(o_mth, train_df)
    for p_record in test_df.to_numpy():
        pred_prob = o_mth.calculate_predictive_prob(*p_record)
        pred_probs.append(pred_prob)

In [44]:
pd.Series(pred_probs).to_csv('../results/new_perturb_pred_probs.csv')

###  Distribution of $\mathbf{r}$

In [62]:
from sklearn.decomposition import PCA
from sklearn.manifold import MDS
from sklearn.manifold import TSNE
dr_class = TSNE
dr_class = PCA

In [86]:
r_viz_mth = copy.deepcopy(init_bssa)
r_viz_fitted_mth = copy.deepcopy(init_bssa)
fit_bayesianssa(r_viz_fitted_mth, real_data_df)

In [87]:
X = np.log(pd.DataFrame(r_viz_mth.r_vecs))
weights = r_viz_fitted_mth.r_dist_param
dr = dr_class(n_components=20, random_state=0)
X_new = dr.fit_transform(X)
pd.DataFrame(X_new).to_csv('../results/pca_r_vecs.csv')
pd.DataFrame(weights).to_csv('../results/weights_for_pca.csv')

#### Covariance

In [48]:
X_arr = X.to_numpy()

In [49]:
term1 = np.zeros((X_arr.shape[1], X_arr.shape[1], X_arr.shape[0]))
for v in range(X_arr.shape[0]):
    tmp_mat = X_arr[v].reshape(-1, 1) @ X_arr[v].reshape(1, -1)
    term1[:, :, v] = tmp_mat
term1 = term1 @ weights

In [50]:
term2 = X.T @ weights
term2 = term2.to_numpy().reshape(-1, 1) @ term2.to_numpy().reshape(1, -1)

In [51]:
r_cov = term1 - term2

In [52]:
pd.DataFrame(r_cov).to_csv('../results/posterior_r_cov.csv')

In [53]:
pd.DataFrame(X_arr).cov().to_csv('../results/prior_r_cov.csv')

In [54]:
r_cov.shape

(127, 127)

###  Confidence values

In [55]:
r_mth = copy.deepcopy(init_bssa)

In [56]:
ex_results = {}
ex_results['target'] = real_data[:, :2]
ex_results['up/down'] = real_data[:, 2]
r_mth.update_distributions(ex_results)
r_mth.run()

In [57]:
with open(f'../results/real_updated_confs.pkl', 'wb') as f:
    pickle.dump(r_mth.confidence, f)

#### Different prior distribution

In [58]:
def random_log_normal(mu_var, sigma_var, size=1):
    r = []
    for i in range(size):
        mu = np.random.normal(0, mu_var)
        sigma = np.exp(np.random.normal(0, sigma_var))
        r.append(np.random.lognormal(mu, sigma))
    return np.array(r)

In [59]:
indices = np.argwhere(nu.T.to_numpy() < 0)
np.random.seed(0)
rln_mth = bayesianssa.BayesianSSA(nu, indices, a, b,
                                  r_dist=random_log_normal, r_dist_param=[1, 1],
                                  n_iter=n_iter, verbose=False)
rln_mth.run()

  0%|          | 0/10000 [00:00<?, ?it/s]

In [60]:
ex_results = {}
ex_results['target'] = real_data[:, :2]
ex_results['up/down'] = real_data[:, 2]
rln_mth.update_distributions(ex_results)
rln_mth.run()

In [61]:
with open(f'../results/real_updated_rln_confs.pkl', 'wb') as f:
    pickle.dump(rln_mth.confidence, f)